# Case study: Classification of shapes
#### Authors: Matteo Caorsi <m.caorsi@l2f.ch>
##### License: TBD


The following notebook explains how to use *giotto* to be able to classify topologically different high-dimensional spaces.

The first step consists in importing the *giotto* library.

In [ ]:
# Importing libraries
import giotto as go
import giotto.datasets as ds
import giotto.time_series as ts
import giotto.diagram as diag
import giotto.homology as hl
from giotto.homology import PersistenceEntropy
import numpy as np
import pandas as pd
import datetime as dt
import sklearn as sk
from sklearn.pipeline import Pipeline
from sklearn.metrics import pairwise_distances

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline

from tqdm import tqdm
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

# Generating orientable surfaces

We are going to consider different classical shapes: the circle, the torus and the sphere.
The purpose of this tutorial is to go thgough the most famous topological spaces and compute their homology groups.

Each of the topological spaces we are going to encounter will be sampled discretely. Aftewards the Vietoris-Rips technique will be applied to the surface and the persistent homology groups will be computed.

In [ ]:
# Representing the circle in 3d with parametric equations.
circle = np.asarray([[np.sin(t),np.cos(t),0] for t in range(400)])
plt.scatter(circle[:,0],circle[:,1])

In [ ]:

# Representing the sphere in 3d with parametric equations
sphere = np.asarray([[np.cos(s)*np.cos(t),np.cos(s)*np.sin(t),np.sin(s)] for t in range(20) for s in range(20)])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(sphere[:,0],sphere[:,1],sphere[:,2])

In [ ]:
# Representing the torus in 3d with parametric equations
torus = np.asarray([[(2+np.cos(s))*np.cos(t),(2+np.cos(s))*np.sin(t),np.sin(s)] for t in range(20) for s in range(20)])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(torus[:,0],torus[:,1],torus[:,2])

In [ ]:
# Saving the results into an array

topological_spaces=np.asarray([circle,sphere,torus])

# Computing persistent homology

In the next section we are using *giotto* to compute the persistent homology groups of the topological spaces we just constructed

In [ ]:

# the homology ranks we choose to consider
homologyDimensions = [0, 1 ,2]
persistenceDiagram = hl.VietorisRipsPersistence(metric='euclidean', max_edge_length=10, homology_dimensions=homologyDimensions, n_jobs=-1)
persistenceDiagram.fit(topological_spaces)

# List of all the time-pordered persistent diagrams obtained from the list of correlation matrices
zDiagrams = persistenceDiagram.transform(topological_spaces)


In [ ]:
print(zDiagrams[0].shape)
print(zDiagrams[1].shape)
print(zDiagrams[2].shape)

# Persistent diagrams

The topological information of the point cloud is synthesised in the persistent diagram. The horizonral axis corresponds to the moment in which an homological generator is born, while the vertical axis corresponds to the moments in which an homological generator dies.
The generators of the homology groups (at given rank) are colored differently

In [ ]:
# plotting the persistent diagram of the circle
fig = plt.figure(figsize=(10,6))
diagram = { dimension: zDiagrams[dimension][0] for dimension in homologyDimensions }

for dimension in homologyDimensions:
    plt.plot(diagram[dimension][:,0], diagram[dimension][:,1], 'o',label='homology '+ str(dimension))
plt.title('Persistent Diagram')
plt.legend(loc='lower right')
plt.plot([0, 2], [0, 2], color='k')
plt.show()

In [ ]:
# plotting the persistent diagram of the sphere
fig = plt.figure(figsize=(10,6))
diagram = { dimension: zDiagrams[dimension][1] for dimension in homologyDimensions }

for dimension in homologyDimensions:
    plt.plot(diagram[dimension][:,0], diagram[dimension][:,1], 'o',label='homology '+ str(dimension))
plt.title('Persistent Diagram')
plt.legend(loc='lower right')
plt.plot([0, 2], [0, 2], color='k')
plt.show()

In [ ]:
# plotting the persistent diagram of the torus
fig = plt.figure(figsize=(10,6))
diagram = { dimension: zDiagrams[dimension][2] for dimension in homologyDimensions }

for dimension in homologyDimensions:
    plt.plot(diagram[dimension][:,0], diagram[dimension][:,1], 'o',label='homology '+ str(dimension))
plt.title('Persistent Diagram for the Torus')
plt.legend(loc='lower right')
plt.plot([0, 2], [0, 2], color='k')
plt.show()

# Generating non-orientable surfaces

We are going to consider different classical shapes: the real projective space and the Klein bottle.
The purpose of the second part of the tutorial is to define shapes via a distance matrix. We also add noise to the distace matrix: the main reason is not to have overlapping points in the persistent diagram.

Each of the topological spaces we are going to encounter will be sampled discretely. Aftewards the Vietoris-Rips technique will be applied to the surface and the persistent homology groups will be computed.

In [ ]:
# computing the adjacency matrix of the grid points, with boundaries identified as in the real projective space
from sklearn.utils.graph_shortest_path import graph_shortest_path

# This functions prepares the grid matrix with boundary identification
def make_matrix(rows, cols):
    n = rows*cols
    M = np.zeros((n,n))
    for r in range(rows):
        for c in range(cols):
            i = r*cols + c
            # Two inner diagonals
            if c > 0: M[i-1,i] = M[i,i-1] = 1 + 0.15*(np.random.rand(1)[0]-0.5)
            # Two outer diagonals
            if r > 0: M[i-cols,i] = M[i,i-cols] = 1 + 0.15*(np.random.rand(1)[0]-0.5)
            # vertical twisted boundary identification
            if c == 0: M[n-i-1,i] = M[i,n-i-1] = 1 + 0.15*(np.random.rand(1)[0]-0.5)
            # horizontal twisted boundary identification
            if r == 0: M[n-i-1,i] = M[i,n-i-1] = 1 + 0.15*(np.random.rand(1)[0]-0.5)
                
    return M

M = make_matrix(20,20)

# computing the distance matrix of the points over the Klein bottle

rp2 = graph_shortest_path(M)

# Plot of the distance matrix
figure = plt.figure(figsize=(10,10))
plt.imshow(rp2)
plt.title('Reciprocal distance between points over the Klein bottle')
plt.colorbar()
plt.show()



In [ ]:
# computing the adjacency matrix of the grid points, with boundaries identified as in the klein bottle
from sklearn.utils.graph_shortest_path import graph_shortest_path

# This functions prepares the grid matrix with boundary identification
def make_matrix(rows, cols):
    n = rows*cols
    M = np.zeros((n,n))
    for r in range(rows):
        for c in range(cols):
            i = r*cols + c
            # Two inner diagonals
            if c > 0: M[i-1,i] = M[i,i-1] = 1 + 0.15*(np.random.rand(1)[0]-0.5)
            # Two outer diagonals
            if r > 0: M[i-cols,i] = M[i,i-cols] = 1 + 0.15*(np.random.rand(1)[0]-0.5)
            # vertical boundary identification
            if c == 0: M[i+cols-1,i] = M[i,i+cols-1] = 1 + 0.15*(np.random.rand(1)[0]-0.5)
            # horizontal twisted boundary identification
            if r == 0: M[n-i-1,i] = M[i,n-i-1] = 1 + 0.15*(np.random.rand(1)[0]-0.5)
                
    return M

M = make_matrix(20,20)

# computing the distance matrix of the points over the Klein bottle

klein = graph_shortest_path(M)

# Plot of the distance matrix
figure = plt.figure(figsize=(10,10))
plt.imshow(klein)
plt.title('Reciprocal distance between points over the Klein bottle')
plt.colorbar()
plt.show()


In [ ]:
# Saving the results into an array

topological_spaces_mat=np.asarray([rp2, klein])

# Computing persistent homology

In the next section we are using *giotto* to compute the persistent homology groups of the topological spaces we just constructed

In [ ]:

# the homology ranks we choose to consider
homologyDimensions = [0, 1 ,2]
persistenceDiagram = hl.VietorisRipsPersistence(metric='precomputed', max_edge_length=10, homology_dimensions=homologyDimensions, n_jobs=-1)
persistenceDiagram.fit(topological_spaces_mat)

# List of all the time-pordered persistent diagrams obtained from the list of correlation matrices
zDiagrams = persistenceDiagram.transform(topological_spaces_mat)


# Persistent diagrams

The topological information of the point cloud is synthesised in the persistent diagram. The horizonral axis corresponds to the moment in which an homological generator is born, while the vertical axis corresponds to the moments in which an homological generator dies.
The generators of the homology groups (at given rank) are colored differently

In [ ]:
# plotting the persistent diagram of the projective space
fig = plt.figure(figsize=(10,6))
diagram = { dimension: zDiagrams[dimension][0] for dimension in homologyDimensions }

for dimension in homologyDimensions:
    plt.plot(diagram[dimension][:,0], diagram[dimension][:,1], 'o',label='homology '+ str(dimension))
plt.title('Persistent Diagram for the RP2')
plt.legend(loc='lower right')
plt.plot([0, 10], [0, 10], color='k')
plt.show()

In [ ]:
# plotting the persistent diagram of the Klein bottle
fig = plt.figure(figsize=(10,6))
diagram = { dimension: zDiagrams[dimension][1] for dimension in homologyDimensions }

for dimension in homologyDimensions:
    plt.plot(diagram[dimension][:,0], diagram[dimension][:,1], 'o',label='homology '+ str(dimension))
plt.title('Persistent Diagram for Klein Bottle')
plt.legend(loc='lower right')
plt.plot([0, 10], [0, 10], color='k')
plt.show()